In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('train.csv')
df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [30]:
unique_labels = df['label'].unique()
print(*sorted(unique_labels, key=lambda s: (s[:3], s[3:])), f'number of types: {len(unique_labels)}', sep='\n')

no_relation
org:alternate_names
org:dissolved
org:founded
org:founded_by
org:member_of
org:members
org:number_of_employees/members
org:place_of_headquarters
org:political/religious_affiliation
org:product
org:top_members/employees
per:alternate_names
per:children
per:colleagues
per:date_of_birth
per:date_of_death
per:employee_of
per:origin
per:other_family
per:parents
per:place_of_birth
per:place_of_death
per:place_of_residence
per:product
per:religion
per:schools_attended
per:siblings
per:spouse
per:title
number of types: 30


In [31]:
from collections import defaultdict
from ast import literal_eval

In [32]:
sub_tkn = 'SUB_ENT'
obj_tkn = 'OBJ_ENT'

def extract(sent, sub_info, obj_info):
    sub_s, sub_e = sub_info['start_idx'], sub_info['end_idx'] + 1
    obj_s, obj_e = obj_info['start_idx'], obj_info['end_idx'] + 1
    
    if sub_s < obj_s:
        sent = sent[:sub_s] + sub_tkn + sent[sub_e:obj_s] + obj_tkn + sent[obj_e:]
    else:
        sent = sent[:obj_s] + obj_tkn + sent[obj_e:sub_s] + sub_tkn + sent[sub_e:]
    return sent

sub_types = set()
obj_types = set()
entities_by_labels = {label: defaultdict(list) for label in unique_labels}
for label, sub_info, obj_info, sent in zip(df['label'], df['subject_entity'], df['object_entity'], df['sentence']):
    sub_info = literal_eval(sub_info)
    obj_info = literal_eval(obj_info)
    entities_by_labels[label]['sub'].append(sub_info['word'])
    entities_by_labels[label]['sub_type'].append(sub_info['type'])
    entities_by_labels[label]['obj'].append(obj_info['word'])
    entities_by_labels[label]['obj_type'].append(obj_info['type'])
    entities_by_labels[label]['sent'].append(extract(sent, sub_info, obj_info))

    sub_types.add(sub_info['type'])
    obj_types.add(obj_info['type'])

print(sub_types, obj_types, sep='\n')

{'ORG', 'PER'}
{'LOC', 'NOH', 'DAT', 'PER', 'ORG', 'POH'}


In [33]:
def gen(sub, sub_type, obj, obj_type, sent: str):
    sub_s = sent.find(sub_tkn)
    obj_s = sent.find(obj_tkn)

    if sub_s < obj_s:
        sent = sent.replace(sub_tkn, sub)
        sub_e = sub_s + len(sub) - 1

        obj_s = sent.find(obj_tkn)
        sent = sent.replace(obj_tkn, obj)
        obj_e = obj_s + len(obj) - 1
    else:
        sent = sent.replace(obj_tkn, obj)
        obj_e = obj_s + len(obj) - 1

        sub_s = sent.find(sub_tkn)
        sent = sent.replace(sub_tkn, sub)
        sub_e = sub_s + len(sub) - 1

    sub_info = {
        'word': sub,
        'start_idx': sub_s,
        'end_idx': sub_e,
        'type': sub_type
    }
    obj_info = {
        'word': obj,
        'start_idx': obj_s,
        'end_idx': obj_e,
        'type': obj_type
    }
    return sent, sub_info, obj_info

In [34]:
import numpy as np

In [35]:
gen_size_aim = 1000

gen_data = {
    'sentence': df['sentence'].tolist(),
    'subject_entity': df['subject_entity'].tolist(),
    'object_entity': df['object_entity'].tolist(),
    'label': df['label'].tolist()
}

tot_size = len(df)
for label, entities in entities_by_labels.items():
    _len = len(entities['sub'])

    gen_size = gen_size_aim - len(entities['sub'])
    if gen_size < 1:
        continue
    tot_size += gen_size
    sub_idxs = np.random.randint(low=0, high=_len, size=gen_size)
    obj_idxs = np.random.randint(low=0, high=_len, size=gen_size)
    sent_idxs = np.random.randint(low=0, high=_len, size=gen_size)

    for i in range(gen_size):
        sent, sub_info, obj_info = gen(
            entities['sub'][sub_idxs[i]],
            entities['sub_type'][sub_idxs[i]],
            entities['obj'][obj_idxs[i]],
            entities['obj_type'][obj_idxs[i]],
            entities['sent'][sent_idxs[i]]
        )
        gen_data['sentence'].append(sent)
        gen_data['subject_entity'].append(sub_info)
        gen_data['object_entity'].append(obj_info)
        gen_data['label'].append(label)

gen_df = pd.DataFrame(gen_data)
ids = [i for i in range(tot_size)]
gen_df = gen_df.sample(frac=1).reset_index(drop=True)
gen_df.insert(loc=0, column='id', value=ids)
gen_df

,id,sentence,subject_entity,object_entity,label
0,0,경기 포천시(시장 박윤국)가 지난 18일 이동작은도서관 리모델링 개관식을 열고 운영...,"{'word': '포천시', 'start_idx': 3, 'end_idx': 5, ...","{'word': '박윤국', 'start_idx': 10, 'end_idx': 12...",org:top_members/employees
1,1,"민관식(閔寬植, 1918년 5월 3일(음력 3월 23일)(음력 3월 23일) ~ 2...","{'word': '민관식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '2006년 1월 16일', 'start_idx': 45, 'end...",per:date_of_death
2,2,"뒤이어 부시 전 미국 대통령과 하토야마 전 일본 총리, 고촉통 전 싱가포르 총리, ...","{'word': '고촉통', 'start_idx': 31, 'end_idx': 33...","{'word': '싱가포르', 'start_idx': 37, 'end_idx': 4...",per:origin
3,3,국토교통부와 한국철도공사에 따르면 오는 30일부터 서울역(용산)∼천안(신창·병점)을...,"{'word': '한국철도공사', 'start_idx': 7, 'end_idx': ...","{'word': '경부선', 'start_idx': 53, 'end_idx': 55...",no_relation
4,4,"3당 합당에 참여하는 것을 두고 민주정의당(전두환 계열)에도 부정적이었고, 통일민주...","{'word': '민주정의당', 'start_idx': 18, 'end_idx': ...","{'word': '전두환', 'start_idx': 24, 'end_idx': 26...",org:top_members/employees
...,...,...,...,...,...
47235,47235,알바니아 민주 한나라당의 초대 대통령인 에베티스 아로니안도 1918년 6월 4일 바...,"{'word': '알바니아', 'start_idx': 0, 'end_idx': 3,...","{'word': '한나라당', 'start_idx': 8, 'end_idx': 11...",org:political/religious_affiliation
47236,47236,긴메이 천황 32년 차남이 비다쓰 천황으로 소쿠이에 올랐다.,"{'word': '비다쓰 천황', 'start_idx': 15, 'end_idx':...","{'word': '긴메이 천황', 'start_idx': 0, 'end_idx': ...",per:parents
47237,47237,"그리스의 카발라 출신으로, 게오르야디스의 가족은 독일 바덴뷔르템베르크주 슈투트가르트...","{'word': '슈투트가르트', 'start_idx': 40, 'end_idx':...","{'word': '바덴뷔르템베르크', 'start_idx': 30, 'end_idx...",no_relation
47238,47238,비토 코를레오네(1891년 12월 7일 ~ 1955년 8월 14일)는 마리오 푸조의...,"{'word': '비토 코를레오네', 'start_idx': 0, 'end_idx'...","{'word': '1955년 8월 14일', 'start_idx': 24, 'end...",per:date_of_death


In [36]:
# gen_df.to_csv('gen_min1k_train.csv', index=False)